In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

In [2]:
full_df = pd.read_csv("grouped-data.csv")
grouped_df = full_df.iloc[:1000]

FileNotFoundError: [Errno 2] No such file or directory: 'grouped-data.csv'

In [ ]:
print(grouped_df.head(1), "\n")
# print(grouped_df.loc[0, ''])

   Unnamed: 0  User-ID        ISBN Year-Of-Publication         Book-Author  \
0           0        2  0195153448                2002  Mark P. O. Morford   

                 Publisher  Book-Rating  
0  Oxford University Press          0.0   



In [ ]:
grouped_df = pd.get_dummies(grouped_df, columns=['Book-Author'])

In [ ]:
X = grouped_df.drop(['User-ID', 'ISBN', 'Book-Rating'], axis=1)
y = grouped_df['Book-Rating']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Assuming df is your DataFrame
# Find the problematic column
problematic_column = None
for column in grouped_df.columns:
    try:
        pd.to_numeric(grouped_df[column])
    except ValueError:
        problematic_column = column
        break

# Convert the problematic column to string
if problematic_column:
    grouped_df[problematic_column] = grouped_df[problematic_column].astype(str)


In [ ]:
# Iterate over each column and check for non-numeric values
for column in grouped_df.columns:
    # Convert column to numeric, coercing errors to NaN
    numeric_column = pd.to_numeric(grouped_df[column], errors='coerce')
    
    # Find non-numeric values
    non_numeric_indices = numeric_column.isnull()
    
    # Check if any non-numeric values are found
    if non_numeric_indices.any():
        # Get the first non-numeric value and print its column header and value
        problematic_value = grouped_df.loc[non_numeric_indices, column].iloc[0]
        print("Column header of the problematic column:", column)
        print("Value found in the entry:", problematic_value)
        break


Column header of the problematic column: ISBN
Value found in the entry: 074322678X


In [ ]:
print(grouped_df.tail(3))

     Unnamed: 0  User-ID        ISBN Year-Of-Publication  \
997         997      507  0312986769                2003   
998         998      507  0312990456                2003   
999         999      507  0312995431                2004   

                   Publisher  Book-Rating  Book-Author_A. C. Gordon  \
997    St. Martin's Minotaur          9.0                     False   
998  St. Martin's Paperbacks          0.0                     False   
999  St. Martin's Paperbacks          0.0                     False   

     Book-Author_A. Manette Ansay  Book-Author_A. S. Byatt  \
997                         False                    False   
998                         False                    False   
999                         False                    False   

     Book-Author_A.S. BYATT  ...  Book-Author_William Eisner  \
997                   False  ...                       False   
998                   False  ...                       False   
999                   False  ... 

In [ ]:
knn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn_model.fit(X_train)

ValueError: could not convert string to float: "St. Martin's Paperbacks"

In [ ]:
def recommend_books(user_features):
    _, indices = knn_model.kneighbors([user_features])
    return grouped_df.iloc[indices[0]][['ISBN', 'Book-Rating']]

In [ ]:
sample_user_features = {
    'Age': 30,
    'Year-Of-Publication': 1995,
    'Location': 'New York',  # Example location
    'Book-Author': 'John Smith',  # Example author
    'Publisher': 'ABC Publishing'  # Example publisher
}

# Preprocess sample user features
sample_location = 'Location_' + sample_user_features['Location'].replace(' ', '_')
sample_author = 'Book-Author_' + sample_user_features['Book-Author'].replace(' ', '_')

In [ ]:
# Create feature vector for the sample user
user_features = [
    sample_user_features['Age'],
    sample_user_features['Year-Of-Publication'],
    0,  # For Publisher (assuming it's not available in sample user data)
    grouped_df.columns.get_loc(sample_location),
    grouped_df.columns.get_loc(sample_author)
]

In [ ]:
# Recommend books for the sample user
recommended_books = recommend_books(user_features)
print("Recommended Books:")
print(recommended_books)